In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from func import *
import jieba
import re
import os
# re -> \d for number, \s for any space (\t\r\n...)

In [2]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
from PIL import Image

In [3]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import itertools as it

import jieba
import re
import numpy as np
from PIL import Image
from func import *

#### 讀取資料夾中的資料

In [4]:
path = 'C:/Users/User/Desktop/python playground/week5/first.txt'
title, data = readdir(path, 0.6)


Number of files : 1


#### 顯示部份資料內容

In [5]:
for i in range(len(data)):
    print('----Document : {0}----'.format(title[i]))
    print(data[i][:50], '...', data[i][-50:-1], sep='\n', end='\n\n')

----Document : f----
1
...


----Document : i----
.
...


----Document : r----
鴻
...


----Document : s----
夏
...


----Document : t----
戀
...




IndexError: string index out of range

#### 新增詞彙至自定義詞庫，並匯入該詞庫

In [ ]:
jieba.load_userdict('./dict.txt')

#### 顯示部分資料內容

#### 新增詞彙至自定義詞庫，並匯入該詞庫

In [ ]:
jieba.load_userdict('./dict.txt')

In [ ]:
stopwords = ['一','二','三','四','五','六','用','淨','利','值','但','中','可','認','我','無','彙',
             '列','將','對','之','率','已','由','佔','本','日','月','年','另','故','於','較',
             '請','再','經','也','在','依','到','等','或','認列','為','按','比','仍','係','後','及',
             '與','以','時','該','的','並','更','註','例如','可能','由於','因此','附註','部分','事項',
             '各位','晶華','品牌','集團','幅度','資產','新','台幣','營業','受到','提供','使用','本年',
             '說明','現金','成本','營業外','收入','結轉','分攤','其他','影響','帳金額','本會','計師',
             '之會計','報表','會計師','財務','詳合','結果','預測','如下','之一','附註','附','註','其中',
             '併','合','因為','查核','報告','性事','增加','會計','相對','仟元','金額','權益','能力',
             '利益','股東','合理性','假設','無形','上','開關鍵','模型','支出','控制','計算','資金',
             '評估','報告','因應','仟元','金額','權益','假設','無形','上','模型','支出','控制','計算',
             '資金','評估','人次','重大','資訊','總額','民國','估計','','一副','公司','年度','包括',
             '正確性','正','確性','減損','共為','預估','預算','維持','公開','整體','表示','特性','市場',
             '科目','發生','年本','重要','判斷','主要','流量','意見','關鍵','同期','評價','正確','地區',
             '總資產','執行','程序','測試','稅後淨利','股份有限公司']
puncs_Full = '！？｡＂＃＄％＆＇（）()＊＋，－／：；＜＝＞＠［＼］$＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.。'
puncs_Half = "!?.#$%&'()*+,-/:;<=>@[]^_{}|~\\\""
savetofile('./stopword.txt', [stopwords])

In [ ]:
stopword = readdir('./stopword.txt')[1].split('\n')

#### 利用 jieba 對文件進行斷字，並統計字詞頻率，顯示前十名資料

In [ ]:
wordDictionary = []
for i in range(len(data)):
    seg_list = jieba.cut(data[i], cut_all=False)
    seg_list = list(seg_list)
    wordDict = {}
    for w in seg_list:
        if (w not in stopword) and (w not in puncs_Full) and (w not in puncs_Half):
            if w in wordDict:
                wordDict[w] = wordDict[w] + 1
            else:
                wordDict[w] = 1
    wordDictionary.append(wordDict)
    print('Number of word in document ({0}) : '.format(title[i]), len(wordDict))
    sort = sorted(wordDict.items(), key=lambda x: x[1], reverse = True)
    print(sort[0:10])
    data[i] = ' '.join(jieba.cut(data[i], cut_all=False))

#### 顯示各文件低於閥值 (limit) 的字詞

In [ ]:
limit = 50
wordsperdocument = 50
for i in range(len(data)):
    count = 0
    print('----Document : {0}----'.format(title[i]))
    for ind, value in wordDictionary[0].items():
        if (value <= limit) & (count != wordsperdocument):
            count += 1
            print(ind, end = ', ')
    print()

#### 利用 wordDictionary 移除只出現一次的字詞，並顯示部份結果

In [ ]:
for i in range(len(data)):
    for key, value in wordDictionary[0].items():
        if value <= limit:
            data[i] = data[i].replace(key, '')
    print('document :', title[i])
    print(data[i][:50], '...', data[i][-50:-1], sep='', end='\n\n')

#### 移除所有標點符號、非中文英文數字之資料

In [ ]:
for i in range(len(data)):
    data[i] = re.sub('[^\u4e00-\u9fff ]', '', data[i])
#[^\u4e00-\u9fff A-Za-z0-9]

#### 印出部份斷字結果

In [ ]:
for i in range(len(data)):
    print('document :', title[i])
    print(data[i][:20], ' ... ', data[i][-20:-1], sep='', end='\n\n')

#### 初始化 TFIDF 物件

In [ ]:
tfidf_vectorizer = TfidfVectorizer(min_df=0.3, analyzer = 'word', stop_words = stopword)
#tfidf_vectorizer = CountVectorizer(stop_words = stopword)

#### 顯示初始化時輸入的停止詞彙

In [ ]:
print(tfidf_vectorizer.get_stop_words())

#### 計算 TFIDF 詞頻矩陣

In [ ]:
tfidf_matrix = tfidf_vectorizer.fit_transform(data)

#### 顯示詞頻矩陣大小

In [ ]:
tfidf_matrix.shape

#### 顯示部份詞頻矩陣中的字詞

In [ ]:
words = tfidf_vectorizer.get_feature_names()
print(words[0:50])

In [ ]:
for i in range(len(data)):
    print('----Document : {0}----'.format(title[i]))
    for j in range(len(words)):
        if tfidf_matrix[i,j] > 0.2:
              print(words[j], tfidf_matrix[i,j])

In [ ]:
dist = cosine_similarity(tfidf_matrix)
print(dist)

In [ ]:
font = r"C:\Windows\Fonts\Microsoft JhengHei UI\msjh.ttc"
alice_mask = np.array(Image.open("week4/cloudy.png"))
wordcloud = WordCloud(font_path=font,background_color="white", mask=alice_mask)
wordcloud.fit_words(wordDict)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud)
plt.savefig("week5/first.png")